# Classification
Classify malware samples according to their labels using their reduced representations.

In [1]:
import tensorflow as tf
print(f"GPU is {'not ' if len(tf.config.list_physical_devices('GPU')) == 0 else ''}available")

2024-05-06 10:07:16.676149: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-06 10:07:17.227343: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU is available


2024-05-06 10:07:18.024944: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-06 10:07:18.066628: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-06 10:07:18.066718: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import keras

# Data Processing

The `1-Data-Exploration.ipynb` file should have been run before this, and, as a result, the `processed-data.csv` file should have been generated.

In [3]:
import pandas as pd

In [4]:
raw_df = pd.read_csv("../data/processed-data.csv")
raw_df

,label,hash,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,TRICKBOT,f16631469eb35406ef4049d30c763cadda571b25bbdb45...,0.234683,0.000000,1.112554,3.848537,10.626454,0.000000,8.636655,4.115375,...,11.081755,0.000044,0.000000,0.0,0.000000,0.0,15.831595,0.000000,7.238762,0.0
1,DARKKOMET,d31a7102cbc54447c251ba62760eb484fd0c9fbb8ea54f...,0.000000,0.000000,5.961174,4.955536,7.219052,6.404585,0.000000,0.000000,...,0.000000,3.986464,15.176069,0.0,1.818527,0.0,19.848658,0.000000,3.754366,0.0
2,SALITY,e7fc7de574f44a966b198b7625bd6c595cad05bd669619...,1.887301,0.000000,3.101377,0.000000,0.000000,0.000000,8.977000,2.093550,...,6.317175,0.000000,0.000000,0.0,0.000000,0.0,13.234521,0.000000,0.323077,0.0
3,VEBZENPAK,4519186b8fb2eaa847255087b44f918928c20e97c2fbea...,0.000000,7.298491,0.000000,0.103305,8.573599,3.445668,8.807044,2.108075,...,6.378553,1.399800,1.832773,0.0,0.000000,0.0,0.000000,2.220947,5.460257,0.0
4,COBALTSTRIKE,dab956e9c864a84d12e8106a24ac3cf2950394152c62b6...,3.745470,1.902894,1.369967,0.000000,0.620541,0.000000,3.691563,1.544375,...,3.513359,5.845166,3.503253,0.0,0.000000,0.0,4.714231,0.000000,12.198085,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,TRICKBOT,020991f3452c6bdac2bd66e60174966bad7b7c201c7907...,0.000000,11.016572,0.000000,1.373428,12.449334,1.808345,9.084649,1.331780,...,10.305679,3.020408,2.662349,0.0,0.000000,0.0,4.298458,1.865488,5.418736,0.0
2819,WAPOMI,1470b463154c299d8fd21b8a449950b9e8aa5aa8765385...,1.578266,0.000000,3.130215,5.973517,10.746815,0.000000,7.970616,11.642600,...,10.740430,0.000000,0.177360,0.0,0.000000,0.0,20.058400,0.000000,9.579028,0.0
2820,MANSABO,78514a632682d1c07ee4f782302bb6a74f2676f1a91b56...,4.159583,0.000000,2.365226,16.249930,8.144193,12.965709,4.145925,0.000000,...,0.000000,0.000000,18.732872,0.0,3.190338,0.0,8.085227,0.000000,0.000000,0.0
2821,TRICKBOT,46401903e85a5c457490a6934ec4dc61fdf28df83af377...,0.000000,2.437977,2.851583,15.546938,13.855932,18.300507,5.672984,0.000000,...,0.000000,0.000000,16.283620,0.0,0.000000,0.0,6.143718,0.000000,0.000000,0.0


For training the model, we don't need the `hash` of the sample.

In [5]:
df = raw_df.drop(columns=["hash"])
df

,label,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,dim-08,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,TRICKBOT,0.234683,0.000000,1.112554,3.848537,10.626454,0.000000,8.636655,4.115375,0.0,...,11.081755,0.000044,0.000000,0.0,0.000000,0.0,15.831595,0.000000,7.238762,0.0
1,DARKKOMET,0.000000,0.000000,5.961174,4.955536,7.219052,6.404585,0.000000,0.000000,0.0,...,0.000000,3.986464,15.176069,0.0,1.818527,0.0,19.848658,0.000000,3.754366,0.0
2,SALITY,1.887301,0.000000,3.101377,0.000000,0.000000,0.000000,8.977000,2.093550,0.0,...,6.317175,0.000000,0.000000,0.0,0.000000,0.0,13.234521,0.000000,0.323077,0.0
3,VEBZENPAK,0.000000,7.298491,0.000000,0.103305,8.573599,3.445668,8.807044,2.108075,0.0,...,6.378553,1.399800,1.832773,0.0,0.000000,0.0,0.000000,2.220947,5.460257,0.0
4,COBALTSTRIKE,3.745470,1.902894,1.369967,0.000000,0.620541,0.000000,3.691563,1.544375,0.0,...,3.513359,5.845166,3.503253,0.0,0.000000,0.0,4.714231,0.000000,12.198085,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,TRICKBOT,0.000000,11.016572,0.000000,1.373428,12.449334,1.808345,9.084649,1.331780,0.0,...,10.305679,3.020408,2.662349,0.0,0.000000,0.0,4.298458,1.865488,5.418736,0.0
2819,WAPOMI,1.578266,0.000000,3.130215,5.973517,10.746815,0.000000,7.970616,11.642600,0.0,...,10.740430,0.000000,0.177360,0.0,0.000000,0.0,20.058400,0.000000,9.579028,0.0
2820,MANSABO,4.159583,0.000000,2.365226,16.249930,8.144193,12.965709,4.145925,0.000000,0.0,...,0.000000,0.000000,18.732872,0.0,3.190338,0.0,8.085227,0.000000,0.000000,0.0
2821,TRICKBOT,0.000000,2.437977,2.851583,15.546938,13.855932,18.300507,5.672984,0.000000,0.0,...,0.000000,0.000000,16.283620,0.0,0.000000,0.0,6.143718,0.000000,0.000000,0.0


Separate into `X` and `y`.

In [6]:
X = df.drop(columns=["label"])
y = df["label"]

In [7]:
print(X.shape)
print(y.shape)

(2823, 32)
(2823,)


# Label Encoding and Train-Test Split

We need to encode the labels for the `y`.

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

We want to save the label encoder for use in the full model.

In [9]:
import pickle

with open("../models/classifier/label-encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

80% of the data will be saved for training, while 20% will be left for testing.

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2258, 32)
(565, 32)
(2258,)
(565,)


# Model Creation

Nothing too special here, we just use several dense layers for classification.

In [12]:
from keras import layers
from keras.models import Sequential

In [13]:
INITIAL_EPOCH = 0

def create_model():
    model = Sequential(name="Classifier")

    model.add(keras.Input((X.shape[1],), name="input"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(len(label_encoder.classes_), activation="softmax"))

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="adam",
        metrics=["sparse_categorical_accuracy"]
    )
    return model

In [14]:
model = create_model()
model.summary()

2024-05-06 10:07:18.380656: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-06 10:07:18.380751: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-06 10:07:18.380784: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-06 10:07:19.295297: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-06 10:07:19.295493: I external/local_xla/xla/stream_executor

Model: "Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 81)             │         5,265 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,465 (158.07 KB)

 Trainable params: 40,465 (158.07 KB)

 Non-trainable params: 0 (0.00 B)

Define callbacks.

In [15]:
import os

checkpoint_path = "../models/classifier/checkpoint.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [16]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1, min_delta=1e-4)

Load latest checkpoint if there is one.

In [17]:
# INITIAL_EPOCH = 54
# model = keras.models.load_model(checkpoint_path)

Train the model.

In [18]:
NUM_EPOCHS = 100

model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    initial_epoch=INITIAL_EPOCH,
    epochs=NUM_EPOCHS,
    callbacks=[checkpointer, early_stopping]
)

Epoch 1/100


I0000 00:00:1714961240.150912 1107248 service.cc:145] XLA service 0x7feffc0060f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714961240.150953 1107248 service.cc:153]   StreamExecutor device (0): Quadro P1000, Compute Capability 6.1
2024-05-06 10:07:20.168822: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-06 10:07:20.979618: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902


38/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.2464 - sparse_categorical_accuracy: 0.1215     

I0000 00:00:1714961242.194317 1107248 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 4.0249 - sparse_categorical_accuracy: 0.1650
Epoch 1: val_loss improved from inf to 2.99107, saving model to ../models/classifier/checkpoint.keras
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - loss: 4.0152 - sparse_categorical_accuracy: 0.1668 - val_loss: 2.9911 - val_sparse_categorical_accuracy: 0.3407
Epoch 2/100
37/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.4998 - sparse_categorical_accuracy: 0.4558 
Epoch 2: val_loss improved from 2.99107 to 2.53307, saving model to ../models/classifier/checkpoint.keras
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.4656 - sparse_categorical_accuracy: 0.4609 - val_loss: 2.5331 - val_sparse_categorical_accuracy: 0.4513
Epoch 3/100
40/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.9229 - sparse_categorical_accuracy: 0.5558 
Epoch 3: val_loss improved from 2.53307 to 2.13944, saving model to ../models/classifier/checkpoint.keras
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.9161 - sparse_categorical_a

Load the best performing model.

In [19]:
model = keras.models.load_model(checkpoint_path)

In [20]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f"Testing loss:      {test_loss:5.5f}")
print(f"Testing accuracy:  {test_acc:5.5f}")

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 1.4837 - sparse_categorical_accuracy: 0.6512
Testing loss:      1.60482
Testing accuracy:  0.63009


In [21]:
model.save("../models/classifier/classifier.keras")